In [ ]:
# 한국어 영어 번역 데이터셋
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import unicodedata

# 실제 프로젝트 AI hub 기타 등등..

# 1.
korean_sentences = [
    "안녕하세요",
    "오늘 날씨가 좋아요",
    "저는 학생입니다",
    "이것은 사과입니다",
    "고양이가 자고 있어요",
    "내일 비가 올까요",
    "저는 커피를 좋아해요",
    "그는 의사입니다",
    "이 책은 재미있어요",
    "우리는 친구예요"
]

english_sentences = [
    "hello",
    "the weather is nice today",
    "i am a student",
    "this is an apple",
    "the cat is sleeping",
    "will it rain tomorrow",
    "i like coffee",
    "he is a doctor",
    "this book is interesting",
    "we are friends"
]

In [3]:
# 텍스트 전처리
# unicode 정규화, 특수문자처리
# NFD  Normalization Form Decomposition -> 분해 가능한 모든 문자를 분해한다
import unicodedata
unicodedata.normalize('NFD',"e'")

"e'"

###### 같은 문장처럼 보이지만 내부적으로 다른 바이트 조합을 갖는 경우 -> 동일한 내부표현으로 바꿔주는 과정

- 유티코드
    - 전 세계의 모든 문자를 하나의 표준 코드로 표현하기 위한 체계

- 유티코드 정규화를 해야되는 이유
    - 문자 하나가 여러 방식으로 표현될 수 있기 때문이다.
    - 그 여러문자들을 하나의 기준으로 통합하고자 사용되고 있다.
###### ucicode_normalizer_md 참고

In [ ]:
# 2.1
def preprocess_sentence(sentence, is_korean=False):
    '''
    Unicode 정규화, 특수문자처리
    Args :
        sentence  : 원본
        is_korean : 한국어 여부
    return :
        전처리된 문장
    '''
    
    sentence = unicodedata.normalize('NFD',sentence)
    if not is_korean:
        sentence = sentence.lower()
    
    # 정규식으로 특수문자 전후에 공백 추가
    # 안녕하세요! -> 안녕하세요 !
    # -> 공백을 추가하는 이유 ^
    # -> 토큰화를 쉽게 하기 위해, 공백을 기준으로 단어분리
    
    # r("[? . ! ,]")
    # 문자중에 ? . ! ,가  나오면 해당문자를 그룹으로 캡처한다. ^
    
    # r" \1 " 캡처한 문자 ( \1 ) 앞뒤로 공백을 하나씩 넣는다.
    sentence = re.sub(r"([? . ! ,])",r" \1 ",sentence)
    sentence = re.sub(r'[" "]'," ",sentence)
    
    # 시작 종료 토큰 추가
    sentence = '<start>' + sentence + '<end>'
    return sentence

In [16]:
korean_processed = [preprocess_sentence(kor) for kor in korean_sentences]
english_processed = [preprocess_sentence(eng) for eng in english_sentences]

In [ ]:
# 문장을 정수 시퀀스로 변환

# 2.2 토크나이저
def create_tokenizer(sentence):
    '''
    단어를 정수 인덱스로 변환
    Vocabulary 구축, word to index mapping
    Args :
        sentence : 문장 리스트
    returns :
        Tokenizer : keras Tokenizer 객체
    '''
    # 토크나이저로 단어사전 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        filters='', # 필터 비활성화
        oov_token='<UNK>'
    )
    # 각 문장(한국어 문장, 영어 문장)으로 단어사전 구축
    tokenizer.fit_on_texts(sentence)
    return tokenizer

In [ ]:
# 한국어 영어 각각의 토크나이저 생성
korean_tokenizer = create_tokenizer(korean_processed)
# 토크나이저를 함수안이 아닌 글로벌 범위에서 생성한경우
english_tokenizer = create_tokenizer(english_preocessed)

# korean_tokenizer.word_index

# 단어사전크기 확인
korean_vocab_size = len(korean_tokenizer.word_index) + 1
english_vocab_size = len(english_tokenizer.word_index) + 1
print(f'한국어 단어사전 크기 : {korean_vocab_size}')
print(f'영어 단어사전 크기 : {english_vocab_size}')

한국어 단어사전 크기 : 25
영어 단어사전 크기 : 30


In [ ]:
# 2.3 점수시퀀스 변경
def encode_sentences(tokenizer, sentences, maxlen):
    '''
    문장을 고정길이의 정수 시퀀스로 변환
    padding : 짧은 문장을 동일길이로 맞추는
    Args :
        tokenizer
        sentence : 전처리된 문장 리스트
        maxlen : 가장 긴 문장의 길이
    Retuens :
        패딩된 점수 시퀀스 배열 tf의 pad_sequnce
    '''
    sequnece = tokenizer.texts_to_sequences(sentences) # [i love you] -> [1,5,7]
    # 길이 맞추기 (패딩 추가)
    padded = tf.keras.preprocessing.sequence.pad_sequences(
        sequnece,
        maxlen = maxlen,
        padding = 'post'
    )
    return padded

#### 패딩 기법 pre / post 어느것을 선택 할지의 대한 기준
- 딥러닝 RNN / LSTM / GUR 전통적인 시계열 처리 기법
    - pre
    - RNN 앞에서부터 읽어 -> 실제 단어 뒤쪽에 몰려있다 -> hidden state 유효한 문맥을 포함 -> h_last에 정보가 압축
- Transformer 계열은 post
    - 위치정보 앞 -> 뒤 순서를 그대로 유지해야 자연스럽다 -> 뒤쪽 패딩은 .. masking 처리 ^


In [ ]:
# 만들어 놓은 함수로 패딩 수행 : encode_sentence
korean_tensor = encode_sentences(korean_tokenizer,korean_processed,max_korean_len)
english_tensor = encode_sentences(english_tokenizer,english_preocessed,max_english_len)

# 최대 시퀀스의 길이 조회 및 변수에 해당 값 저장 (가장 긴 문장기준)
max_korean_len = max(len(s.split()) for s in korean_processed)
max_english_len = max(len(s.split()) for s in english_processed)
print('시퀀스의 최대길이')
print(f'한국어 : {max_korean_len}')
print(f'영어 : {max_english_len}')

print(f'인코딩 첫번째 문장')
print(f'한국어 패딩 문장 : {korean_processed[2]}')
print(f'영어 패딩 문장{english_processed[2]}')
print(f'한국어 패딩 문장 : {korean_tensor[2]}')
print(f'영어 패딩 문장{english_tensor[2]}')

시퀀스의 최대길이
한국어 : 3
영어 : 5
인코딩 첫번째 문장
한국어 패딩 문장 : <start>저는   학생입니다<end>
영어 패딩 문장<start>i   am   a   student<end>
한국어 패딩 문장 : [2 7 0]
영어 패딩 문장[ 4 11  5 12  0]


In [ ]:
BUFFER_SIZE = len(korean_tensor)
BATCH_SIZE = 2

# tensorflow Dataset 객체